In [61]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import random
import BayesOpt
import BayesRTO
import Benoit_Problem

# 0. Data Manipulation
Use ball sampling to sample data to train gaussian process for bayesian optimization. The data is the WLS(weighted least square) between model and actual plant system.

In [62]:
def Ball_sampling(ndim,r_i) -> float:
    '''
    Description:
        This function samples randomly at (0,0) within a ball of radius r_i.
        By adding sampled point onto initial point (u1,u2) you will get 
        randomly sampled points around (u1,u2)

    Arguments:
        ndim: no of dimensions required for sampled point
        r_i: radius from (0,0) of circle area for sampling

    Returns: 
        d_init: sampled distances from (0,0)
    '''
    u      = np.random.normal(0,1,ndim)  
    norm   = np.sum(u**2)**(0.5)
    r      = random.random()**(1.0/ndim)
    d_init = r*u/norm*r_i*2      

    return d_init

def modifier_calc(u,theta,plant_system,model) -> list:

    modifier = plant_system(u) - model(theta,u)

def WLS(theta,U,plant_system:list,model:list) -> float:
    '''
    Description:
        This function finds the sum of weighted least square between
        plant system and model

    Argument: 
        theta: parameter used in model obj func and cons.
        U: sample data 
        plant_system: list containing plant obj func and cons
        model: list containing model obj func and cons

    Returns:
        error: weighted least square between plant and model with input as sample data
    '''
    n_sample = U.shape[0]
    n_function = len(plant_system)
    error = 0

    for i in range(n_sample):
        for j in range(n_function):
            error += (plant_system[j](U[i,:]) - model[j](theta,U[i,:]))**2
            error = error/np.abs(plant_system[j](U[i,:]))

    return error    

def parameter_estimation(U,theta,plant_system,model):
    '''
    Description:
        Uses scipy minimize to find the optimal parameter theta for model that has 
        smallest difference to plant system using input as sample data
    
    Argument:
        theta: parameter used in model obj func and cons.
        U: sample data 
        plant_system: list containing plant obj func and cons
    
    Returns:
        theta_opt: parameter theta that makes minimal difference between plant system 
                    and model using input as sample data
    '''
    sol = minimize(WLS, args=(U,plant_system,model), x0=theta, method='COBYLA')
    return sol.x

def GP_Initialization(n_sample,u_0,theta_0,r,plant_system,model):
    # === first sampling inside the radius === #
    u_dim                           = np.shape(u_0)[0]
    n_function                      = len(plant_system)
    u_sample                        = np.zeros((n_sample,u_dim))
    modifier_sample                 = np.zeros((n_sample,n_function))

    # === Collect Training Dataset (Input) === #
    for sample_i in range(n_sample):
        u_trial                     = u_0 + Ball_sampling(u_dim,r)
        u_sample[sample_i]          = u_trial
    
    # === Estimate the parameters === #
    theta                         = parameter_estimation(u_sample,theta_0,plant_system,model)
    pass


In [63]:
# Test Case
theta = [1,1,1,1]
u_0 = [4,-1]
ndim ,r_i = 2,1
n_s = 1
u_sample = np.zeros((n_s,ndim))

for samplei in range(n_s):
    u_trial = u_0 + Ball_sampling(ndim,r_i)
    u_sample[samplei] = u_trial

plant_system = [Benoit_Problem.Benoit_System_1
                ,Benoit_Problem.con1_system]

model = [Benoit_Problem.Benoit_Model_1,
         Benoit_Problem.con1_Model]

error = WLS(theta,u_sample,plant_system,model)


